<a href="https://colab.research.google.com/github/rhilderbrand/MSDS-422-MachineLearning/blob/main/Copy_of_Hilderbrand_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 5: Principal Component Analysis & Clustering

In [5]:
# import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns

In [6]:
# seed value for random number generators to obtain reproducible results
RANDOM_SEED = 1

In [7]:
#S2 Mount Google Drive to Colab Enviorment
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
#S3 Establish working directory
import os # Operation System
os.getcwd()
%cd /content/gdrive/My Drive/Colab Notebooks/Week 5/
!pwd
!ls
print('Working Directory')
print(os.getcwd())

/content/gdrive/My Drive/Colab Notebooks/Week 5
/content/gdrive/My Drive/Colab Notebooks/Week 5
'Copy of Hilderbrand_Assignment_5'   mnist-original.mat   train.csv
 Hilderbrand_Assignment_5	     test.csv
Working Directory
/content/gdrive/My Drive/Colab Notebooks/Week 5


In [47]:
# Load data
work_dir = " /content/gdrive/My Drive/Colab Notebooks/Week 5/"
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [48]:
train.shape

(42000, 785)

In [49]:
test.shape

(28000, 784)

##1) Random Forest Classifier - All Variables

Begin by fitting a random forest classifier using the full set of 784 explanatory variables and the model training set (train.csv). Record the time it takes to fit the model and then evaluate the model on the test.csv data by submitting to Kaggle.com. Provide your Kaggle.com score and user ID.

In [15]:
X = train.drop(['label'], axis=1)
y = train['label']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [17]:
%%time
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=784)
clf.fit(X_train, y_train)

CPU times: user 3min 6s, sys: 526 ms, total: 3min 6s
Wall time: 3min 7s


In [18]:
%%time
from sklearn.model_selection import cross_val_predict
y_pred_train = cross_val_predict(clf, X_train, y_train)

CPU times: user 12min 5s, sys: 8.75 s, total: 12min 14s
Wall time: 12min 15s


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3279
           1       0.98      0.98      0.98      3744
           2       0.96      0.96      0.96      3342
           3       0.95      0.95      0.95      3478
           4       0.97      0.97      0.97      3243
           5       0.97      0.96      0.96      3064
           6       0.97      0.98      0.98      3337
           7       0.97      0.96      0.97      3551
           8       0.95      0.95      0.95      3217
           9       0.95      0.94      0.94      3345

    accuracy                           0.96     33600
   macro avg       0.96      0.96      0.96     33600
weighted avg       0.96      0.96      0.96     33600



In [20]:
%%time
y_pred_test = cross_val_predict(clf, X_test, y_test)

CPU times: user 2min 30s, sys: 146 ms, total: 2min 30s
Wall time: 2min 30s


In [21]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       853
           1       0.97      0.98      0.98       940
           2       0.94      0.94      0.94       835
           3       0.94      0.92      0.93       873
           4       0.94      0.95      0.95       829
           5       0.94      0.93      0.93       731
           6       0.97      0.96      0.97       800
           7       0.96      0.94      0.95       850
           8       0.94      0.93      0.93       846
           9       0.91      0.93      0.92       843

    accuracy                           0.95      8400
   macro avg       0.95      0.95      0.95      8400
weighted avg       0.95      0.95      0.95      8400



In [22]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred_test)
cf_matrix

array([[838,   0,   1,   0,   1,   2,   4,   0,   7,   0],
       [  0, 923,   6,   6,   0,   2,   2,   0,   1,   0],
       [  4,   1, 788,   9,   7,   0,   7,   8,   9,   2],
       [  3,   4,  20, 800,   3,  16,   1,   8,   9,   9],
       [  0,   0,   4,   1, 789,   0,   5,   3,   2,  25],
       [ 10,   4,   0,  15,   3, 680,   4,   1,   7,   7],
       [  6,   1,   3,   0,   4,  11, 771,   1,   3,   0],
       [  2,   3,  13,   0,   9,   0,   0, 802,   2,  19],
       [  1,   9,   3,  14,   5,  11,   3,   2, 785,  13],
       [  2,   2,   4,   8,  17,   5,   0,  10,  14, 781]])

##2) Principal Component Analysis

Execute principal components analysis (PCA) on the combined training and test set data together, generating principal components that represent 95 percent of the variability in the explanatory variables. The number of principal components in the solution should be substantially fewer than the 784 explanatory variables. Record the time it takes to identify the principal components.

In [50]:
test['label'] = ""

In [51]:
cols=test.columns.tolist()
cols = cols[-1:] + cols[:-1]
test = test[cols]

In [52]:
test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
data = pd.concat([train,test])

In [55]:
X = data.drop(['label'], axis=1)
y = data['label']

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [57]:
%%time
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

CPU times: user 24.7 s, sys: 1.42 s, total: 26.1 s
Wall time: 13.8 s


In [58]:
d

154

##3) PCA and Random Forest Classifier

Using the identified principal components from step (2), use the train.csv to build another random forest classifier. Record the time it takes to fit the model and to evaluate the model on the test.csv data by submitting to Kaggle.com. Provide your Kaggle.com score and user ID.

In [69]:
%%time
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.fit_transform(X_test)

CPU times: user 14.6 s, sys: 4.07 s, total: 18.7 s
Wall time: 9.7 s


In [70]:
X_reduced.shape

(56000, 154)

In [73]:
%%time
clf_2 = RandomForestClassifier(n_estimators=154)
clf_2.fit(X_reduced, y_train)

TypeError: ignored

In [43]:
%%time
y_scores_2 = cross_val_predict(clf_2, X_reduced, y_train)

ValueError: ignored

##4) Model Comparison
Submit both the RF Classifier and the PCA RF Classifier to Kaggle.com, and report both scores along with your user name.  I MUST have your user name to verify submission status.

##5) Rerun Experiment
The experiment we have proposed has a MAJOR design flaw. Identify the flaw. Fix it. Rerun the experiment in a way that is consistent with a training-and-test regimen, and submit this to Kaggle.com. Provide your Kaggle.com score and user ID.

##6) K-Means Clustering
Use k-means clustering to group MNIST observations into 1 of 10 categories and then assign labels.